# 性能テスト結果分析

このノートブックは `performance_test.ipynb` で実行された性能テストの結果を分析し、可視化します。

## 分析内容
1. **データの読み込みと前処理**: `analysis/data` から最新のテスト結果を読み込みます。
2. **全体サマリー**: 全テスト結果の概要を表示します。
3. **サービス間性能比較**: レスポンス時間やスループット(QPS)をサービス間で比較します。
4. **負荷パターン別分析**: 負荷（軽・中・高）による性能の変化を分析します。
5. **スケーラビリティ分析**: 負荷増加に対する各サービスの性能維持能力を評価します。
6. **リソース使用量分析**: コンテナのCPU・メモリ使用量を分析します。
7. **レポート生成**: 分析結果をまとめたMarkdownレポートを生成し、`analysis/report` に保存します。

In [1]:
from pathlib import Path
import glob
import os
import json
from datetime import datetime
import pandas as pd
import numpy as np
from IPython.display import display

# 全体で使うライブラリをまとめてインポートします

import plotly.express as px
import matplotlib.pyplot as plt

In [2]:
# パス設定
BASE_DIR = Path.cwd().parent.parent
DATA_DIR = BASE_DIR / "analysis" / "data"
REPORT_DIR = BASE_DIR / "analysis" / "report"
REPORT_DIR.mkdir(parents=True, exist_ok=True)

print("✅ 必要なライブラリがインポートされました")
print(f"データディレクトリ: {DATA_DIR}")
print(f"レポートディレクトリ: {REPORT_DIR}")

✅ 必要なライブラリがインポートされました
データディレクトリ: /Users/codefox/workspace/practice_infra_arch/key_value_store/analysis/data
レポートディレクトリ: /Users/codefox/workspace/practice_infra_arch/key_value_store/analysis/report


## 1. データの読み込みと前処理

In [3]:
def load_latest_test_data(data_dir):
    """最新の包括的テスト結果(JSON)を読み込む"""
    list_of_files = glob.glob(str(data_dir / 'comprehensive_test_*.json'))
    if not list_of_files:
        print("❌ テスト結果ファイルが見つかりません。")
        return None
    
    latest_file = max(list_of_files, key=os.path.getctime)
    print(f"📂 最新のテスト結果ファイルを読み込み中: {os.path.basename(latest_file)}")
    
    with open(latest_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # エラーを含む結果を除外してDataFrameに変換
    valid_data = [d for d in data if 'error' not in d]
    df = pd.DataFrame(valid_data)
    
    if df.empty:
        print("⚠️ 有効なテストデータがありません。")
        return None
        
    # データ型の整理
    numeric_cols = [
        'clients', 'total_requests', 'successful_requests', 'duration', 
        'error_rate', 'qps', 'avg_response_time', 'median_response_time',
        'p95_response_time', 'p99_response_time', 'min_response_time', 'max_response_time'
    ]
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        
    # 負荷パターンの順序を定義
    df['pattern_name'] = pd.Categorical(df['pattern_name'], categories=['light', 'medium', 'heavy'], ordered=True)
    
    print(f"✅ {len(df)}件の有効なテスト結果を読み込みました。")
    return df

df_results = load_latest_test_data(DATA_DIR)

if df_results is not None:
    df_results.head()

📂 最新のテスト結果ファイルを読み込み中: comprehensive_test_20250912_073050.json
✅ 30件の有効なテスト結果を読み込みました。


## 2. 全体サマリー

In [4]:
if df_results is not None:
    # サマリーテーブルの表示
    summary = df_results.pivot_table(
        index='service_name',
        columns='pattern_name',
        values=['qps', 'avg_response_time', 'error_rate'],
        aggfunc='mean'
    ).fillna(0)
    
    # カラムの整形
    summary.columns = [f'{val}_{cat}' for val, cat in summary.columns]
    
    # 表示形式の調整
    summary_styled = summary.style.format({
        col: '{:.2f}' for col in summary.columns if 'qps' in col or 'avg_response_time' in col
    }).format({
        col: '{:.2f}%' for col in summary.columns if 'error_rate' in col
    }).set_caption("📊 全体パフォーマンスサマリー")
    
    display(summary_styled)
else:
    print("データがありません。")

/var/folders/nc/w7rlvnrs361c069dkdz3nwnw0000gn/T/ipykernel_5604/1001240079.py:3: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  summary = df_results.pivot_table(


,avg_response_time_light,avg_response_time_medium,avg_response_time_heavy,error_rate_light,error_rate_medium,error_rate_heavy,qps_light,qps_medium,qps_heavy
service_name,,,,,,,,,
1-coordinator-ring,4.838633,5.257712,5.372122,0.00%,0.00%,0.00%,204.065891,187.894901,183.909993
10-leaderboard,4.556846,4.407744,4.538003,0.00%,0.00%,0.00%,216.910131,224.168935,217.804016
2-quorum-consistency,6.077150,5.415944,5.824872,0.00%,0.00%,0.00%,162.986701,182.761102,170.066359
3-sharding-replica,5.509165,5.508149,6.518374,0.00%,0.00%,0.00%,179.824031,179.853679,151.758197
4-distributed-lock,6.315332,5.652733,4.155966,0.00%,0.00%,0.00%,156.708367,174.923018,238.121790
5-cache-aside,5.144029,5.518422,6.019304,0.00%,0.00%,0.00%,192.585385,179.521955,164.616037
6-bloom-sstable,3.667425,3.418141,3.457813,0.00%,0.00%,99.60%,269.864954,289.540739,1.023636
7-rate-limiting,4.292150,4.251566,4.249451,0.00%,0.00%,0.00%,230.330784,232.494512,232.704589
8-line-streams,4.387264,3.586648,3.488303,0.00%,0.00%,0.00%,225.491195,275.874064,283.888858


## 3. サービス間性能比較

In [5]:
if df_results is not None:
    fig = px.bar(
        df_results,
        x='service_name',
        y='qps',
        color='pattern_name',
        barmode='group',
        title='🚀 サービス別スループット(QPS)比較',
        labels={'service_name': 'サービス名', 'qps': 'QPS (Queries Per Second)', 'pattern_name': '負荷パターン'},
        category_orders={'pattern_name': ['light', 'medium', 'heavy']}
    )
    fig.update_layout(xaxis_title="サービス名", yaxis_title="QPS (高いほど良い)")
    fig.show()
else:
    print("データがありません。")

In [6]:
if df_results is not None:
    fig = px.bar(
        df_results,
        x='service_name',
        y='avg_response_time',
        color='pattern_name',
        barmode='group',
        title='⏱️ サービス別平均レスポンス時間比較',
        labels={'service_name': 'サービス名', 'avg_response_time': '平均レスポンス時間 (ms)', 'pattern_name': '負荷パターン'},
        category_orders={'pattern_name': ['light', 'medium', 'heavy']}
    )
    fig.update_layout(xaxis_title="サービス名", yaxis_title="平均レスポンス時間 (ms, 低いほど良い)")
    fig.show()
else:
    print("データがありません。")

In [7]:
if df_results is not None:
    df_melted = df_results.melt(
        id_vars=['service_name', 'pattern_name'],
        value_vars=['p95_response_time', 'p99_response_time'],
        var_name='percentile',
        value_name='response_time'
    )
    
    fig = px.bar(
        df_melted,
        x='service_name',
        y='response_time',
        color='percentile',
        facet_col='pattern_name',
        barmode='group',
        title='レスポンス時間パーセンタイル比較 (95%tile, 99%tile)',
        labels={'service_name': 'サービス名', 'response_time': 'レスポンス時間 (ms)', 'percentile': 'パーセンタイル'},
        category_orders={'pattern_name': ['light', 'medium', 'heavy']}
    )
    fig.update_yaxes(matches=None)
    fig.show()
else:
    print("データがありません。")

## 4. スケーラビリティ分析

In [8]:
if df_results is not None:
    fig = px.line(
        df_results.sort_values('pattern_name'),
        x='pattern_name',
        y='qps',
        color='service_name',
        markers=True,
        title='📈 負荷増加に伴うQPSの変化 (スケーラビリティ)',
        labels={'pattern_name': '負荷パターン', 'qps': 'QPS', 'service_name': 'サービス名'}
    )
    fig.update_layout(xaxis_title="負荷パターン", yaxis_title="QPS (高いほど良い)")
    fig.show()
else:
    print("データがありません。")

In [9]:
if df_results is not None:
    fig = px.line(
        df_results.sort_values('pattern_name'),
        x='pattern_name',
        y='avg_response_time',
        color='service_name',
        markers=True,
        title='📉 負荷増加に伴う平均レスポンス時間の変化',
        labels={'pattern_name': '負荷パターン', 'avg_response_time': '平均レスポンス時間 (ms)', 'service_name': 'サービス名'}
    )
    fig.update_layout(xaxis_title="負荷パターン", yaxis_title="平均レスポンス時間 (ms, 低いほど良い)")
    fig.show()
else:
    print("データがありません。")

## 5. リソース使用量分析

注意: この分析は `performance_test.ipynb` でリソース情報が正しく収集されている場合にのみ有効です。

In [10]:
def parse_resource_stats(df):
    """リソース統計情報をパースしてDataFrameに追加

    対応フォーマット:
    - 元の: 'final_stats' 列が辞書 (container_name -> {cpu_percent, memory_usage, ...})
    - CSV向けフラット化: 'final_cpu_avg', 'final_mem_usage_avg' などの数値列
    """
    # 元の辞書形式があれば従来処理を優先
    if 'final_stats' in df.columns:
        cpu_usage = []
        mem_usage = []

        for index, row in df.iterrows():
            stats = row['final_stats']
            if isinstance(stats, dict) and stats:
                # 'app'コンテナの統計を優先
                app_container_key = next((k for k in stats.keys() if 'app' in k), None)
                if app_container_key:
                    cpu_usage.append(stats[app_container_key].get('cpu_percent', np.nan))
                    mem_usage.append(stats[app_container_key].get('memory_usage', np.nan) / (1024*1024)) # MBに変換
                else: # 'app'が見つからない場合は最初のコンテナ
                    first_key = list(stats.keys())[0]
                    cpu_usage.append(stats[first_key].get('cpu_percent', np.nan))
                    mem_usage.append(stats[first_key].get('memory_usage', np.nan) / (1024*1024)) # MBに変換
            else:
                cpu_usage.append(np.nan)
                mem_usage.append(np.nan)

        df['cpu_percent'] = cpu_usage
        df['memory_usage_mb'] = mem_usage
        return df

    # CSVフラット化列に対応
    # 期待する列例: final_cpu_avg, final_cpu_max, final_mem_usage_avg, final_mem_usage_max
    cpu_col_candidates = ['final_cpu_avg', 'final_cpu_max', 'initial_cpu_avg', 'initial_cpu_max']
    mem_col_candidates = ['final_mem_usage_avg', 'final_mem_usage_max', 'initial_mem_usage_avg', 'initial_mem_usage_max']

    cpu_col = next((c for c in cpu_col_candidates if c in df.columns), None)
    mem_col = next((c for c in mem_col_candidates if c in df.columns), None)

    if cpu_col or mem_col:
        # cpu はパーセントのまま、mem はバイト単位であればMBに変換する
        df['cpu_percent'] = df[cpu_col] if cpu_col in df.columns else np.nan
        if mem_col in df.columns:
            # もしすでにMBに変換済みの列名であればそのまま使う。ここでは元がバイトとして扱う。
            df['memory_usage_mb'] = df[mem_col].astype(float) / (1024*1024)
        else:
            df['memory_usage_mb'] = np.nan

        return df

    print("⚠️ リソース統計情報 (final_stats または final_* 列) が見つかりません。")
    return df


if df_results is not None:
    df_resources = parse_resource_stats(df_results.copy())
    if 'cpu_percent' in df_resources.columns:
        display(df_resources[['service_name', 'pattern_name', 'cpu_percent', 'memory_usage_mb']].head())
else:
    print("データがありません。")

,service_name,pattern_name,cpu_percent,memory_usage_mb
0,1-coordinator-ring,light,0.037423,78.171875
1,1-coordinator-ring,medium,0.037974,65.730469
2,1-coordinator-ring,heavy,0.021223,65.554688
3,2-quorum-consistency,light,0.032822,70.683594
4,2-quorum-consistency,medium,0.025129,86.261719


In [11]:
if df_results is not None and 'cpu_percent' in df_resources.columns:
    fig = px.bar(
        df_resources.dropna(subset=['cpu_percent']),
        x='service_name',
        y='cpu_percent',
        color='pattern_name',
        barmode='group',
        title='💻 サービス別CPU使用率',
        labels={'service_name': 'サービス名', 'cpu_percent': 'CPU使用率 (%)', 'pattern_name': '負荷パターン'},
        category_orders={'pattern_name': ['light', 'medium', 'heavy']}
    )
    fig.update_layout(xaxis_title="サービス名", yaxis_title="CPU使用率 (%)")
    fig.show()
else:
    print("CPUデータがありません。")

In [12]:
if df_results is not None and 'memory_usage_mb' in df_resources.columns:
    fig = px.bar(
        df_resources.dropna(subset=['memory_usage_mb']),
        x='service_name',
        y='memory_usage_mb',
        color='pattern_name',
        barmode='group',
        title='🧠 サービス別メモリ使用量',
        labels={'service_name': 'サービス名', 'memory_usage_mb': 'メモリ使用量 (MB)', 'pattern_name': '負荷パターン'},
        category_orders={'pattern_name': ['light', 'medium', 'heavy']}
    )
    fig.update_layout(xaxis_title="サービス名", yaxis_title="メモリ使用量 (MB)")
    fig.show()
else:
    print("メモリデータがありません。")

## 6. レポート生成

In [13]:
def generate_report(df):
    """Markdown形式のレポートを生成（堅牢化）"""
    import math
    if df is None or df.empty:
        return "# 性能テストレポート\n\nテストデータが見つかりませんでした。"

    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    report = f"# 性能テスト分析レポート\n\n生成日時: {timestamp}\n\n"

    report += "## 1. 総合評価\n\n"

    # 高負荷時のQPSランキング（存在すれば）
    heavy_load_df = df[df['pattern_name'] == 'heavy'].sort_values('qps', ascending=False)
    if not heavy_load_df.empty:
        report += "### 高負荷時 QPSランキング\n"
        for _, row in heavy_load_df.iterrows():
            qps = row.get('qps', None)
            qps_str = f"{qps:.2f} QPS" if (qps is not None and not math.isnan(qps)) else "N/A"
            report += f"- **{row['service_name']}**: {qps_str}\n"
        report += "\n"

        report += "### 高負荷時 平均レスポンス時間ランキング (短い順)\n"
        for _, row in heavy_load_df.sort_values('avg_response_time').iterrows():
            art = row.get('avg_response_time', None)
            art_str = f"{art:.2f} ms" if (art is not None and not math.isnan(art)) else "N/A"
            report += f"- **{row['service_name']}**: {art_str}\n"
        report += "\n"
    else:
        report += "⚠️ 高負荷時のデータが見つかりませんでした。\n\n"

    report += "## 2. サマリーテーブル\n\n"
    try:
        summary_df = df.pivot_table(
            index='service_name',
            columns='pattern_name',
            values=['qps', 'avg_response_time']
        ).fillna(0)
        report += summary_df.to_markdown(floatfmt=".2f")
    except Exception as e:
        report += f"サマリーテーブルを作成できませんでした: {e}\n"
    report += "\n\n"

    report += "## 3. 設計パターン別特性分析\n\n"
    for service_name in sorted(df['service_name'].unique()):
        report += f"### {service_name}\n"
        service_df = df[df['service_name'] == service_name]

        # 安全にlight/heavy行を取得する（存在しない場合は代替を使う）
        light_df = service_df[service_df['pattern_name'] == 'light']
        heavy_df = service_df[service_df['pattern_name'] == 'heavy']

        if not light_df.empty:
            light = light_df.iloc[0]
        elif not service_df.empty:
            # lightが無ければclientsが最小の行を代わりに使う
            light = service_df.sort_values('clients').iloc[0]
        else:
            report += "データが不足しています。\n\n"
            continue

        if not heavy_df.empty:
            heavy = heavy_df.iloc[0]
        elif not service_df.empty:
            # heavyが無ければclientsが最大の行を代わりに使う
            heavy = service_df.sort_values('clients', ascending=False).iloc[0]
        else:
            report += "データが不足しています。\n\n"
            continue

        # 安全な数値取得
        def safe_num(val):
            try:
                return float(val)
            except Exception:
                return float('nan')

        light_qps = safe_num(light.get('qps', float('nan')))
        heavy_qps = safe_num(heavy.get('qps', float('nan')))
        light_resp = safe_num(light.get('avg_response_time', float('nan')))
        heavy_resp = safe_num(heavy.get('avg_response_time', float('nan')))

        qps_growth = heavy_qps / light_qps if (light_qps and not math.isnan(light_qps)) else float('nan')
        resp_growth = heavy_resp / light_resp if (light_resp and not math.isnan(light_resp)) else float('nan')

        qps_line = f"軽負荷時 {light_qps:.2f} QPS → 高負荷時 {heavy_qps:.2f} QPS ({qps_growth:.2f}倍)" if (not math.isnan(light_qps) and not math.isnan(heavy_qps) and not math.isnan(qps_growth)) else "スループットデータ不足"
        resp_line = f"軽負荷時 {light_resp:.2f} ms → 高負荷時 {heavy_resp:.2f} ms ({resp_growth:.2f}倍)" if (not math.isnan(light_resp) and not math.isnan(heavy_resp) and not math.isnan(resp_growth)) else "レスポンス時間データ不足"

        report += f"- **スループット**: {qps_line}\n"
        report += f"- **レスポンス時間**: {resp_line}\n"

        # 考察
        if not math.isnan(resp_growth) and resp_growth > 5:
            report += f"- **考察**: ⚠️ 高負荷時にレスポンス時間が大幅に悪化しています。ボトルネックの可能性があります。\n"
        elif not math.isnan(qps_growth) and qps_growth < 2 and (light.get('clients', 0) and heavy.get('clients', 0) and heavy.get('clients', 0)/light.get('clients', 1) > 5):
            report += f"- **考察**: ⚠️ 負荷を増やしてもスループットが伸び悩んでいます。スケーラビリティに課題があるかもしれません。\n"
        else:
            report += f"- **考察**: ✅ 負荷に対して比較的安定した性能を維持しています。\n"

        report += "\n"

    return report


# レポート生成と保存処理
if df_results is not None:
    report_content = generate_report(df_resources if 'df_resources' in globals() else df_results)

    # レポートをファイルに保存
    report_filename = f"performance_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.md"
    report_path = REPORT_DIR / report_filename
    with open(report_path, 'w', encoding='utf-8') as f:
        f.write(report_content)

    print(f"✅ レポートが生成されました: {report_path}")

    # レポートのプレビュー
    print("\n--- レポートプレビュー ---")
    print(report_content[:1000] + "...")
else:
    print("データがないためレポートを生成できません。")


✅ レポートが生成されました: /Users/codefox/workspace/practice_infra_arch/key_value_store/analysis/report/performance_report_20250912_084457.md

--- レポートプレビュー ---
# 性能テスト分析レポート

生成日時: 2025-09-12 08:44:57

## 1. 総合評価

### 高負荷時 QPSランキング
- **9-session-store**: 284.80 QPS
- **8-line-streams**: 283.89 QPS
- **4-distributed-lock**: 238.12 QPS
- **7-rate-limiting**: 232.70 QPS
- **10-leaderboard**: 217.80 QPS
- **1-coordinator-ring**: 183.91 QPS
- **2-quorum-consistency**: 170.07 QPS
- **5-cache-aside**: 164.62 QPS
- **3-sharding-replica**: 151.76 QPS
- **6-bloom-sstable**: 1.02 QPS

### 高負荷時 平均レスポンス時間ランキング (短い順)
- **6-bloom-sstable**: 3.46 ms
- **8-line-streams**: 3.49 ms
- **9-session-store**: 3.49 ms
- **4-distributed-lock**: 4.16 ms
- **7-rate-limiting**: 4.25 ms
- **10-leaderboard**: 4.54 ms
- **1-coordinator-ring**: 5.37 ms
- **2-quorum-consistency**: 5.82 ms
- **5-cache-aside**: 6.02 ms
- **3-sharding-replica**: 6.52 ms

## 2. サマリーテーブル

サマリーテーブルを作成できませんでした: Missing optional dependency 'tabulate'.  

/var/folders/nc/w7rlvnrs361c069dkdz3nwnw0000gn/T/ipykernel_5604/1303113306.py:33: FutureWarning:

The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior

